<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [58]:
import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [60]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [61]:
query_3_1 = '''
SELECT COUNT(*)
FROM public.vacancies
'''

In [62]:
vacancies_count = pd.read_sql_query(query_3_1, connection)
vacancies_count['count'][0]

49197

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [63]:
query_3_2 = '''
SELECT COUNT(*)
FROM public.employers
'''

In [64]:
employers_count = pd.read_sql_query(query_3_2, connection)
employers_count['count'][0]

23501

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [65]:
query_3_3 = '''
SELECT COUNT(*)
FROM public.areas
'''

In [66]:
areas_count = pd.read_sql_query(query_3_3, connection)
areas_count['count'][0]

1362

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [67]:
query_3_4 = '''
SELECT COUNT(*)
FROM public.industries
'''

In [68]:
industries_count = pd.read_sql_query(query_3_4, connection)
industries_count['count'][0]

294

***

- В среднем на каждого работодателя приходится по 2 вакансии.
- Кол-во регионов свидетельствует о том, что ареал поисков не ограничивается только Россией.
- Если сверить кол-во сфер деятельности с реестром росминтруда, можно сказать, что работодатели ищут работников далеко не во всех сферах.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [69]:
# Используем левое присоединение, т.к. нам нужно посчитать именно вакансии
query_4_1 = '''
SELECT 
    ar.name area,
    COUNT(vac.id) cnt
FROM
    public.vacancies vac
    LEFT JOIN public.areas ar ON vac.area_id=ar.id
GROUP BY ar.name
ORDER BY cnt DESC
LIMIT 5
'''

In [70]:
area_vac = pd.read_sql_query(query_4_1, connection)
area_vac

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [71]:
query_4_2 = '''
SELECT 
    COUNT(*)
FROM
    public.vacancies
WHERE
    (salary_from IS NOT NULL) OR (salary_to IS NOT NULL)
'''

In [72]:
cnt_vac_sal = pd.read_sql_query(query_4_2, connection)
cnt_vac_sal['count'][0]

24073

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [73]:
query_4_3 = '''
SELECT 
    ROUND(AVG(salary_from)) sal_from,
    ROUND(AVG(salary_to)) sal_to
FROM
    public.vacancies
'''

In [74]:
avg_sal = pd.read_sql_query(query_4_3, connection)
avg_sal

,sal_from,sal_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [75]:
# Считаем по каждой существующей комбинации schedule и employment
query_4_4 = '''
SELECT 
    schedule,
    employment,
    COUNT(*)
FROM
    public.vacancies
GROUP BY schedule, employment
ORDER BY count DESC
'''

In [76]:
cnt_sh_emp = pd.read_sql_query(query_4_4, connection)
cnt_sh_emp

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [77]:
query_4_5 = '''
SELECT 
    experience "Требуемый опыт работы",
    COUNT(*)
FROM
    public.vacancies
GROUP BY 1
ORDER BY 2
'''

In [78]:
cnt_exp = pd.read_sql_query(query_4_5, connection)
cnt_exp

,Требуемый опыт работы,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [79]:
# Доп. запрос, чтобы узнать сколько ключевых навыков в среднем указывают в вакансиях
query_addit_1 = '''
SELECT
    ROUND(AVG(length(key_skills)-length(replace(key_skills, '\t', '')) + 1), 2)
FROM
    public.vacancies
'''

avg_ks = pd.read_sql_query(query_addit_1, connection)
avg_ks['round'][0]

6.37

In [80]:
# Доп. запрос, чтобы узнать самые высокооплачиваемые сферы деятельности (среди вакансий с указанными зарплатами)
# Левым присоединением добавляем к таблице vac - таблицу ind (для этого нужно еще присоединить связующую таблицу em_ind)
# Столбец salary вычисляется формулой полученной в запросе 6.6
query_addit_2 = '''
SELECT
    ind.name,
    ROUND(AVG((COALESCE(vac.salary_from, vac.salary_to)+COALESCE(vac.salary_to, vac.salary_from)) / 2)) salary
FROM
    public.vacancies vac
    LEFT JOIN public.employers_industries em_ind ON em_ind.employer_id=vac.employer_id
    LEFT JOIN public.industries ind ON ind.id=em_ind.industry_id
WHERE
    (vac.salary_from IS NOT NULL) OR (vac.salary_to IS NOT NULL)
GROUP BY 1
ORDER BY 2 DESC
LIMIT 7
'''

avg_ind = pd.read_sql_query(query_addit_2, connection)
avg_ind

,name,salary
0,"Агрохимия (продвижение, оптовая торговля)",381429.0
1,Агрохимия (производство),339875.0
2,Игорный бизнес,191905.0
3,Корма для животных (производство),168300.0
4,Зоомагазин,167613.0
5,Кадровые агентства,144530.0
6,"Лакокрасочная продукция, сырье (производство)",142803.0


In [81]:
# Доп. запрос, чтобы узнать самые высокооплачиваемые и низкооплачиваемые профессии (среди вакансий с указанными зарплатами)
# Вычисляем salary по каждой профессии.
# 1 таблицу сортируем по убыванию (5 высокооплачиваемых профессий), 2 - по возрастанию (5 низкооплачиваемых). Объединяем через UNION
query_addit_3 = '''
(SELECT
    DISTINCT name,
    ROUND(AVG((COALESCE(salary_from, salary_to)+COALESCE(salary_to, salary_from)) / 2)) salary
FROM
    public.vacancies
WHERE
    (salary_from IS NOT NULL) OR (salary_to IS NOT NULL)
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5)
UNION ALL
(SELECT
    DISTINCT name,
    ROUND(AVG((COALESCE(salary_from, salary_to)+COALESCE(salary_to, salary_from)) / 2)) salary
FROM
    public.vacancies
WHERE
    (salary_from IS NOT NULL) OR (salary_to IS NOT NULL)
GROUP BY 1
HAVING 
    ROUND(AVG((COALESCE(salary_from, salary_to)+COALESCE(salary_to, salary_from)) / 2)) > 5000
ORDER BY 2 ASC
LIMIT 5)
'''

max_min_vac = pd.read_sql_query(query_addit_3, connection)
max_min_vac

,name,salary
0,Брокер по продаже апартаментов,750000.0
1,СТО/Директор по разработке,750000.0
2,FullStack Developer in Unicorn Company,717000.0
3,Frontend Developer in Unicorn Company,717000.0
4,1С:Эксперт по производительности 1С,650000.0
5,Оплачиваемая стажировка в ИТ(не для разработчи...,5301.0
6,Практикант в HR,5832.0
7,Контролер технического состояния ТС (Зяневка),6000.0
8,Менеджер по выпуску одежды,7323.0
9,Помощник (ученик) программиста 1С,7323.0


- Москва с большим отрывом обгоняет остальные города по кол-ву вакансий, что неудивительно, ведь Москва — крупнейший в общероссийском масштабе международный деловой центр.
- У половины вакансий отстутствует информация по зарплатной вилке. Многие работодатели, особенно при найме на важные позиции, не указывают зарплату сразу, чтобы обговорить ее индивидуально с будущим сотрудником.
- В среднем зарплаты находятся в диапазоне от 71 тыс. до 110 тыс. руб.
- Независимо от типа рабочего графика, тип трудоустройства "Полная занятость" всегда преобладает по кол-ву вакансий. Очевидно, работодателям нужен полноценный работник.
- Наиболее часто ищут "джунов" с опытом от 1 года до 3 лет, тогда как наиболее опытных специалистов ищут меньше всего. Компромисс: работодатели ищут людей, которые уже что-то могут и что-то делали, но которые пока не требуют большую зарплату. Низкий спрос на более опытных сотрудников можно объяснить не таким большим кол-вом свободных мест. На такие позиции берут, например, "тимлидов", которые и управляют "джунами". На одного "тимлида" может приходится, к примеру, 10 "джунов".
- В среднем работодатели указывают около 6 ключевых навыков в вакансиях.
- Самой высокооплачиваемой сферой деятельности является "Агрохимия" (среди вакансий с указанными зарплатами). Довольно узкоспециализированная, но очень важная сфера, в которой ощущается дефицит кадров, что выливается в хорошие зарплаты.
- Самыми высокооплачивыми профессиями являются разработчики, директора и брокеры по продаже аппартаментов. Самыми низкооплачиваемыми позициями являются практиканты и стажеры. Ничего удивительного, ИТ-специалисты и управляющие позиции всегда отличались космическими зарплатами, тогда как "зеленые" студенты-практиканты идут работать скорее за опытом и знаниями.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [82]:
query_5_1 = '''
SELECT 
    em.name,
    COUNT(vac.id) cnt
FROM
    public.vacancies vac
    LEFT JOIN public.employers em ON vac.employer_id=em.id
GROUP BY em.name
ORDER BY cnt DESC
LIMIT 5
'''

In [83]:
cnt_emp = pd.read_sql_query(query_5_1, connection)
cnt_emp

,name,cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [84]:
# Areas присоединяем через FULL, чтобы мы владели информацией о регионах, которые не попали в вакансии.
# Затем для каждого региона присоединяем информацию о работодателях (LEFT JOIN).
# Кол-во вакансий сортируем по возрастанию, кол-во работодателей - по убыванию. 1 позиция - наш регион.
query_5_2 = '''
SELECT 
    ar.name,
    COUNT(vac.id) count_vacancies,
    COUNT(em.id) count_employers
FROM
    public.vacancies vac
    FULL JOIN public.areas ar ON vac.area_id=ar.id
    LEFT JOIN public.employers em ON ar.id=em.area
GROUP BY ar.name
ORDER BY count_vacancies, count_employers DESC
LIMIT 1
'''

In [85]:
area_unemployed = pd.read_sql_query(query_5_2, connection)
area_unemployed['name'][0]

'Россия'

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [86]:
# Используем COUNT(DISTINCT), чтобы подсчитать кол-во именно уникальных регионов (т.к. мы присоединили все таблицы к vac, а не ar)
query_5_3 = '''
SELECT 
    em.name,
    COUNT(DISTINCT ar.name) count_areas
FROM
    public.vacancies vac
    LEFT JOIN public.areas ar ON vac.area_id=ar.id
    LEFT JOIN public.employers em ON vac.employer_id=em.id
GROUP BY em.name
ORDER BY count_areas DESC
LIMIT 5
'''

In [87]:
emp_by_area = pd.read_sql_query(query_5_3, connection)
emp_by_area

,name,count_areas
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [88]:
# Считаем только тех работодателей, которые не указаны в связующей таблице em_in
query_5_4 = '''
SELECT 
    COUNT(*)
FROM
    public.employers em
    LEFT JOIN public.employers_industries em_in ON em.id=em_in.employer_id
WHERE em_in.employer_id IS NULL
'''

In [89]:
emp_no_ind = pd.read_sql_query(query_5_4, connection)
emp_no_ind['count'][0]

8419

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [90]:
# Считаем тех работодателей, у которых есть ровно 4 записи в связующей таблице em_in
query_5_5 = '''
SELECT 
    em.name
FROM
    public.employers em
    LEFT JOIN public.employers_industries em_in ON em.id=em_in.employer_id
GROUP BY em.name
HAVING COUNT(em_in.industry_id) = 4
ORDER BY em.name
OFFSET 2
LIMIT 1
'''

In [91]:
emp_with_ind = pd.read_sql_query(query_5_5, connection)
emp_with_ind['name'][0]

'2ГИС'

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [92]:
query_5_6 = '''
SELECT 
    COUNT(*)
FROM
    public.employers em
    LEFT JOIN public.employers_industries em_in ON em.id=em_in.employer_id
    LEFT JOIN public.industries ind ON ind.id=em_in.industry_id
WHERE ind.name = 'Разработка программного обеспечения'
'''

In [93]:
emp_develop = pd.read_sql_query(query_5_6, connection)
emp_develop['count'][0]

3553

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [94]:
# Цикл проходится по каждой строке таблицы (стартуем со второй строки, т.к. первая - загаловки столбцов)
# Добавляем названия городов в список million_cities до тех пор, пока цикл не встречает город Волгоград (он последний в нашей таблице)
# С помощью try-except захватываем любые ошибки, т.к. иногда код ругается на пустые значения
import requests
from bs4 import BeautifulSoup

url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')

million_cities = []

for row in page.find_all('tr')[1:]:
    try:
        city = row.find_all('td')[1].find('a').text
        million_cities.append(city)
        if city == 'Волгоград':
            break
    except Exception:
        continue

In [95]:
# Сначала создаем подзапрос, чтобы затем использовать результат в 1 и 2 таблице UNION
# Считаем кол-во вакансий Яндекса для каждого уникального региона, который содержится в нашем списке million_cities
# Присоединяем с помощью UNION суммированный столбец cnt
query_5_7 = f'''
with yandex as (SELECT 
    DISTINCT ar.name,
    COUNT(vac.id) cnt
FROM
    public.vacancies vac
    LEFT JOIN public.employers em ON em.id=vac.employer_id
    LEFT JOIN public.areas ar ON ar.id=vac.area_id
WHERE 
    (em.name = 'Яндекс') AND (ar.name IN {tuple(million_cities)})
GROUP BY 1
ORDER BY 2)
SELECT 
    name,
    cnt
FROM yandex
UNION ALL
SELECT
    'Total',
    SUM(cnt)
FROM yandex
'''

In [96]:
emp_develop = pd.read_sql_query(query_5_7, connection)
emp_develop

,name,cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Казань,25.0
5,Пермь,25.0
6,Ростов-на-Дону,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


***

In [97]:
# Доп. запрос, чтобы узнать самых "требовательных" крыпных работодателей, которые разместили 200 и более вакансий
# Немного модифицируем формулу, полученную в запросе 6.5
# Т.к. в key_skills могут быть пропуски, для таких случаев преобразуем нашу формулу так, чтобы она давала 0 (0-1+1=0)
query_addit_4 = '''
SELECT
    em.name,
    ROUND(AVG(length(coalesce(vac.key_skills, ''))-length(replace(coalesce(vac.key_skills, '1\t'), '\t', '')) + 1), 2) "Кол-во требуемых навыков"
FROM
    public.vacancies vac
    LEFT JOIN public.employers em ON em.id=vac.employer_id
GROUP BY 1
HAVING
    COUNT(vac.id) >= 200
ORDER BY 2 DESC
'''

ks_by_emp = pd.read_sql_query(query_addit_4, connection)
ks_by_emp

,name,Кол-во требуемых навыков
0,Совкомбанк Технологии,5.24
1,Ростелеком,5.08
2,Газпром нефть,5.08
3,МТС,4.83
4,DataArt,4.64
5,Тинькофф,4.33
6,ИК СИБИНТЕК,3.26
7,СБЕР,2.89
8,Яндекс,0.35


In [98]:
# Доп. запрос для подсчета кол-ва работодателей, которым требуются люди с опытом
query_addit_5 = '''
SELECT 
    COUNT(DISTINCT em.name)
FROM
    public.vacancies vac
    LEFT JOIN public.employers em ON vac.employer_id=em.id
WHERE
    vac.experience != 'Нет опыта'
ORDER BY 1
LIMIT 5
'''

cnt_unjun_emp = pd.read_sql_query(query_addit_5, connection)
cnt_unjun_emp['count'][0]

13182

- "Яндекс" бесспорный лидер по кол-ву размещенных вакансий. "Яндекс" одна из крупнейших IT-компаний в России, которая развивает сразу несколько бизнес-направлений: портальные сервисы, электронная коммерция, сервисы онлайн-заказа такси и заказа еды и т.д. На все эти направления требуется огромное кол-во сотрудников, чем и обуславливается огромное кол-во размещенных вакансий.
- В справочнике о регионах имеются записи о целых странах, но работодатели не выставляют такой регион вакансии даже если работа удаленная. Работодателям важно, чтобы сотрудник находился в относительной близости от какого-нибудь головного офиса компании.
- Треть работодателей не указывают сферу деятельности. Это именно та проблема, с которой hh.ru нужно бороться, ибо указание сферы деятельности упрощает поиск нужных вакансий соискателям.
- Почти половина тех, кто указывает, имеют сферу деятельности "Разработка ПО". В век информационных технологий разработчики - одна из самых ценных профессий, которая будет только набирать популярность.
- Лидером по кол-ву требуемых навыков среди крупных работодателей является Совкомбанк Технологии.
- Большая половина работодателей не ищут людей без опыта. Работодателям нужен работник с каким-либо опытом работы в своей сфере.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [99]:
query_6_1 = '''
SELECT
    COUNT(*)
FROM
    public.vacancies
WHERE
    (lower(name) LIKE '%data%') OR (lower(name) LIKE '%данн%')
'''

In [100]:
count_vac_ds = pd.read_sql_query(query_6_1, connection)
count_vac_ds['count'][0]

1771

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [101]:
query_6_2 = '''
SELECT
    COUNT(*)
FROM
    public.vacancies
WHERE
    ((lower(name) LIKE '%data scientist%')
    OR (lower(name) LIKE '%data science%')
    OR (lower(name) LIKE '%исследователь данных%')
    OR ((name LIKE '%ML%') AND (name NOT LIKE '%HTML%'))
    OR (lower(name) LIKE '%machine learning%')
    OR (lower(name) LIKE '%машинн%обучен%'))
    AND
    ((lower(name) LIKE '%junior%')
    OR (experience = 'Нет опыта')
    OR (employment = 'Стажировка'))
'''

In [102]:
count_vac_jds = pd.read_sql_query(query_6_2, connection)
count_vac_jds['count'][0]

51

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [103]:
query_6_3 = '''
SELECT
    COUNT(*)
FROM
    public.vacancies
WHERE
    ((lower(name) LIKE '%data scientist%')
    OR (lower(name) LIKE '%data science%')
    OR (lower(name) LIKE '%исследователь данных%')
    OR ((name LIKE '%ML%') AND (name NOT LIKE '%HTML%'))
    OR (lower(name) LIKE '%machine learning%')
    OR (lower(name) LIKE '%машинн%обучен%'))
    AND
    ((lower(key_skills) LIKE '%sql%')
    OR (lower(key_skills) LIKE '%postgres%'))
'''

In [104]:
count_vac_sql = pd.read_sql_query(query_6_3, connection)
count_vac_sql['count'][0]

201

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [105]:
query_6_4 = '''
SELECT
    COUNT(*)
FROM
    public.vacancies
WHERE
    ((lower(name) LIKE '%data scientist%')
    OR (lower(name) LIKE '%data science%')
    OR (lower(name) LIKE '%исследователь данных%')
    OR ((name LIKE '%ML%') AND (name NOT LIKE '%HTML%'))
    OR (lower(name) LIKE '%machine learning%')
    OR (lower(name) LIKE '%машинн%обучен%'))
    AND
    (lower(key_skills) LIKE '%python%')
'''

In [106]:
count_vac_py = pd.read_sql_query(query_6_4, connection)
count_vac_py['count'][0]

351

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [107]:
# Кол-во '\t' + 1 - наше кол-во ключевых навыков, т.к. они все разделены именно знаком табуляции
# Считаем разницу через length(), причем от изначального признака отнимаем тот же признак, но с удаленными '\t'
# Получаем кол-во '\t', затем считаем среднее
query_6_5 = '''
SELECT
    ROUND(AVG(length(key_skills)-length(replace(key_skills, '\t', '')) + 1), 2)
FROM
    public.vacancies
WHERE
    (lower(name) LIKE '%data scientist%')
    OR (lower(name) LIKE '%data science%')
    OR (lower(name) LIKE '%исследователь данных%')
    OR ((name LIKE '%ML%') AND (name NOT LIKE '%HTML%'))
    OR (lower(name) LIKE '%machine learning%')
    OR (lower(name) LIKE '%машинн%обучен%')
'''

In [108]:
avg_ds_ks = pd.read_sql_query(query_6_5, connection)
avg_ds_ks['round'][0]

6.41

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [109]:
# Зарплата вычисляется как (salary_from+salary_to) / 2, если все вилки указаны
# Если хотя бы одна вилка отсутствует, то с помощью функции coalesce() - мы заменяем отсутствующий признак на имеющийся
# Затем считаем средний по опыту
query_6_6 = '''
SELECT
    experience,
    ROUND(AVG((COALESCE(salary_from, salary_to)+COALESCE(salary_to, salary_from)) / 2))
FROM
    public.vacancies
WHERE
    ((lower(name) LIKE '%data scientist%')
    OR (lower(name) LIKE '%data science%')
    OR (lower(name) LIKE '%исследователь данных%')
    OR ((name LIKE '%ML%') AND (name NOT LIKE '%HTML%'))
    OR (lower(name) LIKE '%machine learning%')
    OR (lower(name) LIKE '%машинн%обучен%'))
    AND
    ((salary_from IS NOT NULL) OR (salary_to IS NOT NULL))
GROUP BY 1
ORDER BY 1
'''

In [110]:
avg_salary = pd.read_sql_query(query_6_6, connection)
avg_salary

,experience,round
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

In [111]:
# Доп. запрос, чтобы узнать сколько всего вакансий относятся к DS
query_addit_6= '''
SELECT
    COUNT(*)
FROM
    public.vacancies
WHERE
    ((lower(name) LIKE '%data scientist%')
    OR (lower(name) LIKE '%data science%')
    OR (lower(name) LIKE '%исследователь данных%')
    OR ((name LIKE '%ML%') AND (name NOT LIKE '%HTML%'))
    OR (lower(name) LIKE '%machine learning%')
    OR (lower(name) LIKE '%машинн%обучен%'))
'''

cnt_ds = pd.read_sql_query(query_addit_6, connection)
cnt_ds['count'][0]

480

In [112]:
# Доп. запрос, чтобы узнать на какие позиции в DS-вакансиях требуются специалисты с опытом "Более 6 лет"
query_addit_7 = '''
SELECT
    DISTINCT name,
    salary_from,
    salary_to
FROM
    public.vacancies
WHERE
    ((lower(name) LIKE '%data scientist%')
    OR (lower(name) LIKE '%data science%')
    OR (lower(name) LIKE '%исследователь данных%')
    OR ((name LIKE '%ML%') AND (name NOT LIKE '%HTML%'))
    OR (lower(name) LIKE '%machine learning%')
    OR (lower(name) LIKE '%машинн%обучен%'))
    AND
    (experience = 'Более 6 лет')
'''

teamlead_ds = pd.read_sql_query(query_addit_7, connection)
teamlead_ds

,name,salary_from,salary_to
0,Data Scientist (Python),None,None
1,Lead Data Scientist,None,None
2,Machine learning research lead,None,None
3,Machine Learning Team Lead,None,None
4,Machine Learning Team Lead (Go/Scala/Python) (...,None,None
5,ML developer,None,None
6,Senior Data Scientist,None,None
7,Senior ML-инженер,None,None
8,Team Lead Data Science,None,None
9,Архитектор решений с опытов в Tiny ML,None,None


In [113]:
# Доп. запрос для определения лидеров по кол-ву размещенных DS-вакансий
query_addit_8 = '''
SELECT
    em.name,
    COUNT(vac.id)
FROM
    public.vacancies vac
    LEFT JOIN public.employers em ON em.id=vac.employer_id
WHERE
    ((lower(vac.name) LIKE '%data scientist%')
    OR (lower(vac.name) LIKE '%data science%')
    OR (lower(vac.name) LIKE '%исследователь данных%')
    OR ((vac.name LIKE '%ML%') AND (vac.name NOT LIKE '%HTML%'))
    OR (lower(vac.name) LIKE '%machine learning%')
    OR (lower(vac.name) LIKE '%машинн%обучен%'))
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
'''

ds_emp = pd.read_sql_query(query_addit_8, connection)
ds_emp

,name,count
0,СБЕР,37
1,Bell Integrator,25
2,Банк ВТБ (ПАО),18
3,VK,15
4,Positive Technologies,11
5,Яндекс,9
6,EvenBet Gaming,9
7,МегаФон,8
8,Andersen,7
9,Ozon,6


In [114]:
# Доп. запрос для определения среднего диапазона зарплат для DS-специалистов
query_addit_9 = '''
SELECT 
    ROUND(AVG(salary_from)) sal_from,
    ROUND(AVG(salary_to)) sal_to
FROM
    public.vacancies
WHERE
    ((lower(name) LIKE '%data scientist%')
    OR (lower(name) LIKE '%data science%')
    OR (lower(name) LIKE '%исследователь данных%')
    OR ((name LIKE '%ML%') AND (name NOT LIKE '%HTML%'))
    OR (lower(name) LIKE '%machine learning%')
    OR (lower(name) LIKE '%машинн%обучен%'))
'''

ds_sal = pd.read_sql_query(query_addit_9, connection)
connection.close()
ds_sal

,sal_from,sal_to
0,146805.0,216356.0


- ~4% вакансий имеют отношение к данным и ~1% имеет отношение непосредственно к области DS. И этот процент будет только расти, поскольку данные - чуть ли не самый важный аспект в бизнесе.
- ~10% от вакансий DS составляют специалисты уровня Junior.
- В ~42% вакансий DS требуют знание SQL и в ~73% вакансий - знание Python. Это самые базовые навыки, которые должен иметь в своем арсенале DS-соискатель. Python - самый популярный язык программирования для машинного обучения и анализа данных, а SQL - уже давно стал стандартом для работы с данными.
- В среднем специалистам DS требуются примерно столько же ключевых навыков (6.41), сколько и для остальных профессий (6.37). 
- В среднем зарплаты для DS-специалистов находятся в диапазоне от 147 тыс. до 216 тыс. руб. Это намного выше диапазона по всем вакансиям, что объясняется дефицитом кадров в этой сфере.
- С повышением опыта в сфере DS - зарплата в среднем возрастает на 80.5%. Также среди вакансий DS с требуемым опытом "Более 6 лет" - зарплату не указывают. Как правило на такие позиции требуются Тимлиды и Сеньоры.
- СБЕР активнее всех разыскивает DS-специалистов, что неудивительно для крупнейшего финансового конгломерата в России и Восточной Европе. Банковская система - самая нуждающаяся в услугах DS-специалистов сфера.

# Общий вывод по проекту

Целевым признаком для нас является **зарплатная вилка**. Если при указании верхней и нижней границы - можно просто посчитать среднее, то для случаев, когда вилка не указывается вообще (половина всех вакансий), нужно строить отдельную модель, которая по остальным параметрам будет формировать приблизительное значение зарплаты, чтобы отображать более подходящие рекомендации.

Решение задачи с одной зарплатной вилкой недостаточно. Треть работодателей не указывают сферу деятельности. Чтобы построить более адекватную систему рекомендаций, нужно решить эту задачу. Например, сделать поле со сферой деятельности, при регистрации вакансии на сайте, обязательной для заполнения. В отличие от зарплаты, нет причин не указывать этот пункт в вакансии.

Предположительно все данные могут пригодиться нам при построении будущей модели:
- *регион* - чтобы определить, что работодатель распологается в нужном месте для соискателя
- *тип трудоустройства*, *тип рабочего графика* - чтобы найти соискателю нужные ему условия
- *ключевые навыки* - чтобы выдать соискателю вакансии "по его уровню"
- *и т.д.*

Интересные замечания по всем вакансиям:
- Москва - лидер-регион по кол-ву вакансий
- У половины вакансий отсутствует информация о зарплате
- В среднем зарплаты находятся в диапазоне от 71 тыс. до 110 тыс. руб.
- Требуемый опыт "от 1 года до 3 лет" - лидер по кол-ву вакансий
- В среднем работодатели указывают 6 ключевых навыков
- "Яндекс" - лидер-компания по кол-ву вакансий
- Работодатели не выставляют в качестве региона целую страну
- Треть работодателей не указывают сферу деятельности
- Большая половина работодателей не ищут людей без опыта работы

Наша сфера - DS. Несмотря на довольно малый процент от всех вакансий на сайте (1%), это одна из самых быстроразвивающихся сфер, кадровый голод в которой ощущается уже сейчас (выражается в огромных зарплатах по сравнению с остальными вакансиями).

Интересные замечания по DS-вакансиям:
- СБЕР - лидер-компания по кол-ву вакансий
- В среднем DS-специалистам требуются столько же ключевых навыков, сколько и для остальных профессий (6)
- С повышением опыта - зарплата возрастает в среднем на 80.5%
- Одни из самых "профитных" навыков DS-специалиста - SQL и Python
- В среднем зарплаты находятся в диапазоне от 147 тыс. до 216 тыс. руб.